In [ ]:
import warnings
from math import log

import astropy.coordinates as coord
from astropy.constants import G as _G
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('apw-notebook')
%matplotlib inline
from scipy.optimize import root
import emcee

import gala.potential as gp
from gala.units import UnitSystem
from scipy.special import loggamma

import starfish

In [ ]:
units = UnitSystem(u.mpc, u.yr, u.Msun, u.rad)
G = _G.decompose(units).value

In [ ]:
n_data = 512
Mbh = 4E6
gamma = 3.5
a = 2. # mpc
b = 1000. # mpc = 1 pc

# Maximum energy to consider is that of a circular orbit at r = 2 mpc
r_min = 2

pot = gp.KeplerPotential(m=Mbh*units['mass'], units=units)

In [ ]:
def density(r, gamma, a, b):
    return a**gamma * r**-gamma

def potential(r, Mbh):
    return G*Mbh / r

def rv_to_E(r, v, Mbh):
    return -0.5*v**2 + potential(r, Mbh)

def v_c(r, Mbh):
    return np.sqrt(G * Mbh / r)

def log_df(E, L, Mbh, gamma, beta):
    g = gamma
    
    num = -2*beta*log(L) + (g - beta-1.5) * log(E) + beta*log(2)
    den = 1.5*log(2*np.pi) + (g - 2*beta) * log(G*Mbh)
    gams = loggamma(g - 2*beta + 1) - loggamma(g - beta - 0.5)

    return (num - den + gams).real

max_E = rv_to_E(r_min, v_c(r_min, Mbh), Mbh)
max_E

# Sample from the DF using MCMC

In [ ]:
def lnprob(p, Mbh, gamma):
    ln_r, ln_v = p
    r = np.exp(ln_r)
    v = np.exp(ln_v)
    
    E = rv_to_E(r, v, Mbh)
    
    if not 0 < E < max_E:
        return -np.inf
    
    if ln_r < log(r_min):
        return -np.inf
    
    df_ = log_df(E, 1., Mbh, gamma, 0.) # HACK: isotropic
    
    if not np.isfinite(df_):
        return -np.inf
    
    return df_ + 2*ln_r + 2*ln_v + ln_r + ln_v

In [ ]:
p0 = np.zeros((32, 2))
p0[:,0] = np.random.normal(5., 0.1, size=p0.shape[0])
p0[:,1] = v_c(p0[:,0], Mbh)
p0 = np.log(p0)

In [ ]:
sampler = emcee.EnsembleSampler(nwalkers=p0.shape[0], dim=2, 
                                lnpostfn=lnprob, args=(Mbh, gamma))

In [ ]:
_ = sampler.run_mcmc(p0, 16384)

In [ ]:
for dim in range(p0.shape[1]):
    plt.figure()
    for walker in sampler.chain[...,dim]:
        plt.plot(walker, marker='', drawstyle='steps-mid', alpha=0.2)

In [ ]:
flatchain = np.vstack(sampler.chain[:,500::8])
r_samples, v_samples = np.exp(flatchain.T)

In [ ]:
bins = np.logspace(-1, 3, 64)
bin_ctr = (bins[:-1] + bins[1:]) / 2.
V = 4*np.pi*bin_ctr**2 * (bins[1:] - bins[:-1])

H,_ = np.histogram(r_samples, bins)

plt.plot(bin_ctr, H/V)
plt.plot(bin_ctr, density(bin_ctr, gamma, r_min, bin_ctr.max()))
plt.plot(bin_ctr, bin_ctr**-1.2)

plt.xscale('log')
plt.yscale('log')

In [ ]:
plt.hist(v_samples, bins='auto');
plt.xlabel('$v$')## Now get 3D coordinates assuming isotropy:

## Now get 3D coordinates assuming isotropy:

In [ ]:
import gala.integrate as gi

In [ ]:
# further downsample:
r_subset = r_samples[::64]
v_subset = v_samples[::64]

w0 = starfish.rv_to_3d_isotropic(r_subset*units['length'], 
                                 v_subset*units['speed'])

orbit = pot.integrate_orbit(w0, dt=1E-2, n_steps=10000,
                            Integrator=gi.DOPRI853Integrator)
w = orbit[-1].represent_as(coord.PhysicsSphericalRepresentation)

In [ ]:
_ = orbit[:,1].plot()

Compare initial and final density profile:

In [ ]:
Hi,_ = np.histogram(r_subset, bins)
Hf,_ = np.histogram(w.r.value, bins)

plt.plot(bin_ctr, density(bin_ctr, gamma, r_min, bin_ctr.max()))
plt.plot(bin_ctr, Hi/V, marker='', linestyle='--')
plt.plot(bin_ctr, Hf/V, marker='', linestyle='--')

plt.xscale('log')
plt.yscale('log')

In [ ]:
np.save('rv.npy', np.vstack((r_samples, v_samples)))

## Test emcee sample gamma

In [ ]:
def log_df_arr(E, L, Mbh, gamma, beta):
    g = gamma
    
    num = -2*beta*np.log(L) + (g - beta - 1.5) * np.log(E) + beta*np.log(2)
    den = 1.5*np.log(2*np.pi) + (g - 2*beta) * np.log(G*Mbh)
    gams = loggamma(g - 2*beta + 1) - loggamma(g - beta - 0.5)

    return (num - den + gams).real

def lnprob_gamma(p, Mbh, r, v):
    gamma = p[0]
    
    E = rv_to_E(r, v, Mbh)
    
    if not 0.5 < gamma < 5.:
        return -np.inf
    
    df_ = log_df_arr(E, 1., Mbh, gamma, 0.) # HACK: isotropic
    
    if not np.all(np.isfinite(df_)):
        return -np.inf
    
    return np.sum(df_ + 2*np.log(r) + 2*np.log(v))

In [ ]:
p0 = np.zeros((32, 1))
p0[:,0] = np.random.normal(gamma, 0.1, size=p0.shape[0])

In [ ]:
sampler = emcee.EnsembleSampler(nwalkers=p0.shape[0], dim=p0.shape[1], 
                                lnpostfn=lnprob_gamma, args=(Mbh, r_samples[:64], v_samples[:64]))

In [ ]:
_ = sampler.run_mcmc(p0, 1024)

In [ ]:
for dim in range(p0.shape[1]):
    plt.figure()
    for walker in sampler.chain[...,dim]:
        plt.plot(walker, marker='', drawstyle='steps-mid', alpha=0.2)